# Notebook #1
## This is an example notebook is creating the UDF to send table change events within Snowflake to AWS EventBridge
### UDF is 'vectorized' for scalability and performance
Written by Steven.Maser@snowflake.com

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

#Variables for object creation (define to your needs)
DB= 'EVENT_MONITORING'
SCHEMA = 'PUBLIC'
SUB_SCHEMA = 'PROCESSING'
AWS_REGION = 'us-west-2'
AWS_CLIENT = 'events'
AWS_EVENTBUS_NAME='default' # used 'default' for Demonstrations
AWS_ROLE_ARN='arn:aws:iam::xxxxxxxxxx:role/my-external-access-iam' # provided by your AWS acount
EXT_UDF_NAME = 'notifyEventBridge' # Name of UDF this notebook creates

In [ ]:
--1. Create Network Rule, listing allowed AWS Services
CREATE OR REPLACE NETWORK RULE {{DB}}.{{SCHEMA}}.aws_event_network_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('events.{{AWS_REGION}}.amazonaws.com');


In [ ]:
--2.  Create Security Integration (defined in AWS and the Role to Assume with EventBridge permissions)
CREATE SECURITY INTEGRATION if not exists aws_event_security_integration
 TYPE = API_AUTHENTICATION
 AUTH_TYPE = AWS_IAM
 ENABLED = TRUE
 AWS_ROLE_ARN = '{{AWS_ROLE_ARN}}'; -- Use ARN of role you will assume

In [ ]:
--3.  Get (USER_ARN and External_ID to add a trust in AWS Policy )
describe integration aws_event_security_integration;

--4. Add value of API_AWS_EXTERNAL_ID to your AWS Role's Trust Policy
      ---https://docs.snowflake.com/en/sql-reference/external-functions-creating-aws-common-api-integration-proxy-link#set-up-the-trust-relationship-s-between-snowflake-and-the-new-iam-role

### Step 4:  Use value API_AWS_EXTERNAL_ID value above to set your Trust Policy in AWS for security

Warning: will change values if you re-create your security integration

In [ ]:
--5.  Create an AWS Secret and grant usage
CREATE OR REPLACE SECRET {{DB}}.{{SCHEMA}}.aws_event_token
  TYPE = CLOUD_PROVIDER_TOKEN
  API_AUTHENTICATION = aws_event_security_integration;
GRANT READ ON SECRET {{DB}}.{{SCHEMA}}.aws_event_token TO ROLE {{session.get_current_role()}};

In [ ]:
--6.  Create External Access Integration connecting steps 1-4 and grant access
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION aws_event_integration
  ALLOWED_NETWORK_RULES = ({{DB}}.{{SCHEMA}}.aws_event_network_rule)
  ALLOWED_AUTHENTICATION_SECRETS = ({{DB}}.{{SCHEMA}}.aws_event_token)
  ENABLED = true;
GRANT USAGE ON INTEGRATION aws_event_integration TO ROLE {{session.get_current_role()}};

In [ ]:
--7. Create UDF that batches 10 records at a time to send to AWS EventBridge (its current limit)
        --- Note:  total event size must be less than 256KB
        --another example:  https://medium.com/snowflake/making-batch-api-calls-in-snowflake-with-vectorized-udfs-b9a15c7c0704

CREATE OR REPLACE FUNCTION {{DB}}.{{SCHEMA}}.{{EXT_UDF_NAME}}(event string)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
HANDLER = 'put'
PACKAGES=('snowflake-snowpark-python','boto3','botocore')
EXTERNAL_ACCESS_INTEGRATIONS = (aws_event_integration)
SECRETS = ('aws_event_token' = aws_event_token)
AS
$$
import pandas
import _snowflake
from _snowflake import vectorized
import boto3

class Boto3Client:
    _client = None
    
    @classmethod
    def _get_client(cls):
        if cls._client is not None:
            return cls._client
            
        cloud_provider_object = _snowflake.get_cloud_provider_token("aws_event_token")
        aws_access_key_id = cloud_provider_object.access_key_id
        aws_secret_access_key = cloud_provider_object.secret_access_key
        aws_session_token = cloud_provider_object.token
        
        cls._client = boto3.client(
            '{{AWS_CLIENT}}',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token,
            region_name='{{AWS_REGION}}'
        )
        return cls._client

@vectorized(input=pandas.DataFrame, max_batch_size=10)
def put(event):
    eventbridge = Boto3Client._get_client()

    payload = event[0].apply(lambda x: {
        'Source': 'Snowflake|db.schema.table',
        'DetailType': 'Table Change Event',
        'Detail': x,
        'EventBusName': '{{AWS_EVENTBUS_NAME}}'
    }).tolist() 

    response = eventbridge.put_events(Entries=payload) 
    data = []
    for record in response['Entries']:
        data.append(record['EventId'])
    return pandas.Series(data)
$$;

In [ ]:
--8. Test Creating Payload
select OBJECT_CONSTRUCT('Account',current_account(),'Region',current_region(),'Database','$db_name','Schema','$schema','Table','$table','Record',OBJECT_CONSTRUCT(*))::STRING as Event from TEST_TABLE;

In [ ]:
--9.  Test Sending Events to AWS EventBridge and return AWS EventIds
SELECT {{DB}}.{{SCHEMA}}.{{EXT_UDF_NAME}}(
    OBJECT_CONSTRUCT('Account',current_account(), 'Region',current_region(), 'Database','$db_name', 'Schema','$schema', 'Table','$table', 'Record',OBJECT_CONSTRUCT(*))::STRING
) as EventId FROM TEST_TABLE;
